# Code Generator

We use Frontier models to generate high performance C++ code from Python code. The are best ranked models from the SEAL coding leaderboard.


In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [37]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [38]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
GEMINI_MODEL = "gemini-2.0-flash"

# Low cost models:
# OPENAI_MODEL = "gpt-4o-mini"
# CLAUDE_MODEL = "claude-3-haiku-20240307"

In [4]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [5]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [7]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [8]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [9]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [56]:
def optimize_gemini(python):
    model = google.generativeai.GenerativeModel(
        model_name=GEMINI_MODEL,
        system_instruction=system_message,
        generation_config={"response_mime_type": "text/plain"},
    )
    
    response = model.generate_content(user_prompt_for(python), stream=True)

    result = ""
    for chunk in response:
        piece = ""
        if getattr(chunk, "text", None) and chunk.text: #check if text is not empty
            piece = chunk.text
        elif getattr(chunk, "candidates", None) and chunk.candidates:
            parts = getattr(chunk.candidates[0].content, "parts", [])
            piece = "".join(getattr(p, "text", "") for p in parts if getattr(p, "text", ""))

        if piece:
            result += piece
            print(piece, end="", flush=True)

    write_output(result)
    return result

In [46]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [11]:
exec(pi)

Result: 3.141592658589
Execution Time: 13.470583 seconds


In [12]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <chrono>
#include <iomanip>

// Function to perform the calculation
double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        int j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    // Record start time
    auto start_time = std::chrono::high_resolution_clock::now();

    // Perform calculations
    double result = calculate(100000000, 4, 1) * 4;

    // Record end time
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end_time - start_time;

    // Print result and execution time
    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << elapsed.count() << " seconds" << std::endl;

    return 0;
}
```

In [64]:
optimize_gemini(pi)

'```cpp\n#include <iostream>\n#include <iomanip>\n#include <chrono>\n\ndouble calculate(long long iterations, int param1, int param2) {\n    double result = 1.0;\n    for (long long i = 1; i <= iterations; ++i) {\n        double j = static_cast<double>(i) * param1 - param2;\n        result -= (1.0 / j);\n        j = static_cast<double>(i) * param1 + param2;\n        result += (1.0 / j);\n    }\n    return result;\n}\n\nint main() {\n    auto start_time = std::chrono::high_resolution_clock::now();\n    double result = calculate(100000000, 4, 1) * 4;\n    auto end_time = std::chrono::high_resolution_clock::now();\n\n    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);\n    double execution_time = static_cast<double>(duration.count()) / 1000000.0;\n\n    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;\n    std::cout << std::fixed << std::setprecision(6) << "Execution Time: " << execution_time << " seconds"

In [ ]:
!clang++ -O3 -std=c++17 -o optimized optimized.cpp
!./optimized

# Compiling C++ and executing

This next cell contains the command to compile a C++ file on a Mac.  
It compiles the file `optimized.cpp` into an executable called `optimized`  
Then it runs the program called `optimized`

In [14]:
# Compile C++ and run the executable (as a terminal command with !)

!clang++ -O3 -std=c++17 -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.359214117000 seconds


In [15]:
optimize_claude(pi)

#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    #pragma omp parallel for reduction(-:result)
    for (long long i = 1; i <= iterations; ++i) {
        double j = i * static_cast<double>(param1) - param2;
        result -= 1.0 / j;
        j = i * static_cast<double>(param1) + param2;
        result += 1.0 / j;
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100'000'000, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;

    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);
    std::cout << "Execution Time: " << duration.count() / 1e6 << " seconds" << std::endl;

    return 0;
}

In [16]:
# Repeat for Claude 

!clang++ -O3 -std=c++17 -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.358252000000 seconds


In [17]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [18]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 69.605668 seconds


In [21]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <functional>
#include <chrono>
#include <limits>

// LCG generator
uint32_t lcg(uint32_t &value, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 1u << 32) {
    value = (a * value + c) % m;
    return value;
}

// Finds the maximum subarray sum
int64_t max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    std::vector<int> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        uint32_t value = lcg(seed);
        random_numbers[i] = value % (max_val - min_val + 1) + min_val;
    }
    
    int64_t max_sum = std::numeric_limits<int64_t>::min();
    for (int i = 0; i < n; ++i) {
        int64_t current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    return max_sum;
}

// Calculates the total maximum subarray sum for 20 iterations
int64_t total_ma

In [ ]:
!clang++ -O3 -std=c++17 -o optimized optimized.cpp
!./optimized

optimized.cpp:9:94: warning: shift count >= width of type [-Wshift-count-overflow]
uint32_t lcg(uint32_t &value, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 1u << 32) {
                                                                                             ^  ~~
1 warning generated.
Total Maximum Subarray Sum (20 runs): 0
Execution Time: 0.882276 seconds


In [25]:
optimize_claude(python_hard)

#include <iostream>
#include <vector>
#include <chrono>
#include <limits>
#include <iomanip>

using namespace std;
using namespace chrono;

class LCG {
private:
    uint64_t value;
    const uint64_t a = 1664525;
    const uint64_t c = 1013904223;
    const uint64_t m = 1ULL << 32;

public:
    LCG(uint64_t seed) : value(seed) {}

    uint64_t next() {
        value = (a * value + c) % m;
        return value;
    }
};

int64_t max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    vector<int64_t> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg.next() % (max_val - min_val + 1) + min_val;
    }

    int64_t max_sum = numeric_limits<int64_t>::min();
    int64_t current_sum = 0;
    int64_t min_sum = 0;

    for (int i = 0; i < n; ++i) {
        current_sum += random_numbers[i];
        max_sum = max(max_sum, current_sum - min_sum);
        min_sum = min(min_sum, current_sum);
    }

    return max_sum;
}

int64_t tot

In [26]:
!clang++ -O3 -std=c++17 -o optimized optimized.cpp
!./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.001078 seconds


In [27]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [28]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [68]:
def stream_gemini(python):
    stream = google.generativeai.GenerativeModel(
        model_name=GEMINI_MODEL,
        system_instruction=system_message,
        generation_config={"response_mime_type": "text/plain"}
    )
    response = stream.generate_content(user_prompt_for(python), stream=True)
    result = ""
    for chunk in response:
        # For streaming, content is directly in `chunk.text` or `chunk.candidates[0].content.parts.text`
        if hasattr(chunk, 'text') and chunk.text:
            result+= chunk.text or ""
            yield result
        elif hasattr(chunk, 'candidates') and chunk.candidates:
            parts = getattr(chunk.candidates[0].content, "parts", [])
            piece = "".join(getattr(p, "text", "") for p in parts if getattr(p, "text", ""))
            if piece:
                result += piece
                yield result


In [69]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_gemini(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [70]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

In [31]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [32]:
def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [33]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [72]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)